In [1]:
import duckdb
# duckdb.read_csv("example.csv")                # read a CSV file into a Relation
# duckdb.read_parquet("example.parquet")        # read a Parquet file into a Relation
# duckdb.read_json("example.json")              # read a JSON file into a Relation

# duckdb.sql("SELECT * FROM 'example.csv'")     # directly query a CSV file
# duckdb.sql("SELECT * FROM 'example.parquet'") # directly query a Parquet file
# duckdb.sql("SELECT * FROM 'example.json'")    # directly query a JSON file

In [2]:
con = duckdb.connect("file.db")

In [6]:
con.read_json("./io_test_data.json", format = 'unstructured')

┌──────────┬────────────┬──────────────────────────────────────────────────────────────────────────────────────────────┐
│ deviceId │ timestamp  │                                           metrics                                            │
│  int64   │   int64    │                  struct("name" varchar, tagindex bigint, tagvalue double)[]                  │
├──────────┼────────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│     5880 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 9464.665648947313}, {'name': A2, 'tagIndex': 2, '…  │
│     2416 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 707.8517778206106}, {'name': A2, 'tagIndex': 2, '…  │
│     1566 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 9512.683448648495}, {'name': A2, 'tagIndex': 2, '…  │
│     5935 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 8121.450749149209}, {'name': A2, 'tagIndex': 2, '…  │
│     7902 │ 1723170332 │ [{'nam

In [7]:
sql = "SELECT * FROM './io_test_data.json'"
con.sql(sql)

┌──────────┬────────────┬──────────────────────────────────────────────────────────────────────────────────────────────┐
│ deviceId │ timestamp  │                                           metrics                                            │
│  int64   │   int64    │                  struct("name" varchar, tagindex bigint, tagvalue double)[]                  │
├──────────┼────────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│     5880 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 9464.665648947313}, {'name': A2, 'tagIndex': 2, '…  │
│     2416 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 707.8517778206106}, {'name': A2, 'tagIndex': 2, '…  │
│     1566 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 9512.683448648495}, {'name': A2, 'tagIndex': 2, '…  │
│     5935 │ 1723170332 │ [{'name': A1, 'tagIndex': 1, 'tagValue': 8121.450749149209}, {'name': A2, 'tagIndex': 2, '…  │
│     7902 │ 1723170332 │ [{'nam

In [22]:
# SELECT json_transform(j, '{"family": "TINYINT", "coolness": "DECIMAL(4, 2)"}') FROM example;

sql = "SELECT deviceId, timestamp, UNNEST(metrics) as metric_item FROM './io_test_data.json'"
con.sql(sql)

┌──────────┬────────────┬─────────────────────────────────────────────────────────────────┐
│ deviceId │ timestamp  │                             metric                              │
│  int64   │   int64    │    struct("name" varchar, tagindex bigint, tagvalue double)     │
├──────────┼────────────┼─────────────────────────────────────────────────────────────────┤
│     5880 │ 1723170332 │ {'name': A1, 'tagIndex': 1, 'tagValue': 9464.665648947313}      │
│     5880 │ 1723170332 │ {'name': A2, 'tagIndex': 2, 'tagValue': 7445.954394495071}      │
│     5880 │ 1723170332 │ {'name': A3, 'tagIndex': 3, 'tagValue': 4981.610671865472}      │
│     5880 │ 1723170332 │ {'name': A4, 'tagIndex': 4, 'tagValue': 4278.567188264892}      │
│     5880 │ 1723170332 │ {'name': A5, 'tagIndex': 5, 'tagValue': 8564.825901347784}      │
│     5880 │ 1723170332 │ {'name': A6, 'tagIndex': 6, 'tagValue': 5289.177111915427}      │
│     5880 │ 1723170332 │ {'name': A7, 'tagIndex': 7, 'tagValue': 3109.580237037

In [27]:
sql = "select deviceId, timestamp, unnest(metric_item) from (SELECT deviceId, timestamp, UNNEST(metrics) as metric_item FROM './io_test_data.json')"
con.sql(sql)

┌──────────┬────────────┬─────────┬──────────┬────────────────────┐
│ deviceId │ timestamp  │  name   │ tagIndex │      tagValue      │
│  int64   │   int64    │ varchar │  int64   │       double       │
├──────────┼────────────┼─────────┼──────────┼────────────────────┤
│     5880 │ 1723170332 │ A1      │        1 │  9464.665648947313 │
│     5880 │ 1723170332 │ A2      │        2 │  7445.954394495071 │
│     5880 │ 1723170332 │ A3      │        3 │  4981.610671865472 │
│     5880 │ 1723170332 │ A4      │        4 │  4278.567188264892 │
│     5880 │ 1723170332 │ A5      │        5 │  8564.825901347784 │
│     5880 │ 1723170332 │ A6      │        6 │  5289.177111915427 │
│     5880 │ 1723170332 │ A7      │        7 │ 3109.5802370370106 │
│     5880 │ 1723170332 │ A8      │        8 │ 7208.7505718467355 │
│     5880 │ 1723170332 │ A9      │        9 │  8517.361056392974 │
│     5880 │ 1723170332 │ A10     │       10 │  4282.319365783478 │
│       ·  │      ·     │  ·      │        · │  

In [30]:
# PIVOT Cities
# ON Year IN (2000, 2010)
# USING sum(Population)
# GROUP BY Country;

sql = """
CREATE TABLE device_iot_01 AS
PIVOT (
    select deviceId, timestamp, unnest(metric_item) from 
        (SELECT deviceId, timestamp, UNNEST(metrics) as metric_item FROM '../data/io_test_data.json')
)
ON name
USING first(tagValue)
GROUP BY deviceId, timestamp
"""

con.sql(sql)

In [26]:
con.sql("SELECT unnest({'a': 42, 'b': 84});")

┌───────┬───────┐
│   a   │   b   │
│ int32 │ int32 │
├───────┼───────┤
│    42 │    84 │
└───────┴───────┘

In [31]:
sql = """
select * from device_iot_01
"""
con.sql(sql)


┌──────────┬────────────┬────────────────────┬───┬────────────────────┬────────────────────┬────────────────────┐
│ deviceId │ timestamp  │         A1         │ … │        A97         │        A98         │        A99         │
│  int64   │   int64    │       double       │   │       double       │       double       │       double       │
├──────────┼────────────┼────────────────────┼───┼────────────────────┼────────────────────┼────────────────────┤
│     4404 │ 1723170332 │  9517.432054526838 │ … │ 160.64571717921194 │ 3786.4163403832385 │  4319.004590433276 │
│     9930 │ 1723170332 │  6730.674911890311 │ … │  5893.709011263243 │  4513.011239819236 │ 3871.7841386314235 │
│     1692 │ 1723170332 │   8342.33560917053 │ … │ 338.76559766456535 │  7863.826905875481 │ 4476.6836211069185 │
│     5556 │ 1723170332 │  864.7845424211165 │ … │  4270.303369826512 │  6839.437938838715 │  5715.543266455876 │
│     1624 │ 1723170332 │  9687.112461252838 │ … │ 3874.6003290131316 │  5037.1252700820

In [4]:
con.close()

In [3]:
sql = """
COPY (select * from device_iot_01) TO 'device_iot_01.csv' (HEADER, DELIMITER ',');
"""
con.sql(sql)